In [4]:
import os


In [5]:
%pwd


'k:\\pipeline - Copy\\research'

In [6]:
os.chdir("../")


In [7]:
%pwd


'k:\\pipeline - Copy'

In [8]:

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [10]:

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

In [10]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [11]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [13]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
except Exception as e:
    raise e

[2025-09-09 13:03:21,373: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-09 13:03:21,375: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-09 13:03:21,375: INFO: common: created directory at: artifacts]
[2025-09-09 13:03:21,376: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2025-09-09 13:03:21,377: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]


In [ ]:
# checkpoint = tf.keras.callbacks.ModelCheckpoint("best_model.h5", save_best_only=True)
# tensorboard = tf.keras.callbacks.TensorBoard(log_dir="logs")

# model.fit(
#     x_train, y_train,
#     validation_data=(x_val, y_val),
#     epochs=20,
#     callbacks=[checkpoint, tensorboard]
# )

""" for this we need the checkpoint object and tensorboard object  to train the model , so when we train the model in next  pipeline we can use this callback list """

In [ ]:
# for this do all the modular stulls except giving it as astarting point to run from pipeline or main.py , because we are not directly going to run this file , instead we will call it when we do training from pipeline or main.py , so we will not add this to main.py or pipeline.py now